In [9]:
### Si données locales
u.user <- read.csv(file='u.user.csv', sep='|', header=T)
u.item <- read.csv(file='u.item.csv', sep='|', header=T)
u.data <- read.csv(file='u.data.csv', sep='|', header=T)
library(Matrix)
m.sparse <- sparseMatrix(u.data[,1],u.data[,2],x=u.data[,3])
rownames(m.sparse) <- paste('u', 1:nrow(m.sparse), sep='')
colnames(m.sparse) <- paste('i', 1:ncol(m.sparse), sep='')
mae <- function(m1, m2) mean(abs(m1 - m2), na.rm=T)

m <- as.matrix(m.sparse)                # transforme en matrice ordinaire
m.na <- m
m.na[m.na==0] <- NA
mae.rmse <- function(m, m.hat) {
    return(list(mae=mean(abs(m - m.hat), na.rm=T), rmse=sqrt(mean((m - m.hat)^2, na.rm=T))))
}
new.m.mae.rmse <- function(dimRedu){
# matrice diagonale pour réduire les matrices originales
m.d2 <- diag(c(m.svd$d[1:dimRedu],rep(0,length(m.svd$d)-dimRedu)))
# nouvelle matrice basée sur les deux premières dimensions uniquement
m.svd2 <- round((m.svd$u %*% m.d2 %*% t(m.svd$v)),2)
mae.rmse(m.svd2, m)
    }

In [10]:
# merge users and ratings
names(u.user)<- c('user.id','age','gender','job')
u.user <- u.user[,c(-5)]
cat.age <- function(x) floor(x/20)
u.user[, 2] <- sapply(u.user[, 2], cat.age)
for (i in 2:4){
u.user[, i] <- sapply(u.user[, i], as.factor)
}
head(u.user)
names(u.data)<- c('user.id','movie.id','rating')
u.data <- u.data[,c(-4)]
u.data[, 3] <- sapply(u.data[, 3], as.factor)
head(u.data)
u.item <- u.item[,c(-2,-3,-4,-5)]
for (i in 2:20){
u.item[, i] <- sapply(u.item[, i], as.factor)
}
head(u.item)
u.r <- merge(u.user, u.data, by = "user.id")
# merge userRating with movie to have actual movie titles instead of ids
u.r.v <- merge(u.r, u.item, by = "movie.id")
head(u.r.v)

user.id,age,gender,job
1,1,M,technician
2,2,F,other
3,1,M,writer
4,1,M,technician
5,1,F,other
6,2,M,executive


user.id,movie.id,rating
196,242,3
186,302,3
22,377,1
244,51,2
166,346,1
298,474,4


movie.id,unknown,Action,Adventure,Animation,Children.s,Comedy,Crime,Documentary,Drama,Fantasy,Film.Noir,Horror,Musical,Mystery,Romance,Sci.Fi,Thriller,War,Western
1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
6,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


movie.id,user.id,age,gender,job,rating,unknown,Action,Adventure,Animation,...,Fantasy,Film.Noir,Horror,Musical,Mystery,Romance,Sci.Fi,Thriller,War,Western
1,918,2,M,scientist,3,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,684,1,M,student,4,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,660,1,M,student,3,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,339,1,M,lawyer,5,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,699,2,M,other,3,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,94,1,M,student,4,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [11]:
library('e1071')

In [12]:
formula <- as.factor(rating) ~ .

In [13]:
new.m.cv <- function(dimRedu){
# matrice diagonale pour réduire les matrices originales
m.d2 <- diag(c(m.svd$d[1:dimRedu],rep(0,length(m.svd$d)-dimRedu)))
# nouvelle matrice basée sur les deux premières dimensions uniquement
m.svd2 <- round((m.svd$u %*% m.d2 %*% t(m.svd$v)),2)
mae(m.svd2[i.test.b], m.na[i.test.b])
    }
to.plot <- c()

In [14]:
i.observed <- dim(u.r.v)[1]
set.seed(5)
i.hasard <- sample(seq(1,i.observed,1), i.observed)
fold.size <- round(length(i.hasard) / 5)
i.false <- rep(FALSE, dim(u.r.v)[1])
for (i in 1:5){
fold.number <- i

## Index booléen pour les cellules de test et d'entraînement
i.test.b <- i.false
## Les cellules indexées du replis correspondant sont fixées à TRUE pour le test...
i.test.b[ i.hasard[((fold.number-1) * fold.size):((fold.number) * fold.size)] ] <- TRUE
    

## ...et à FALSE pour l'entraînement
i.train.b <-  !i.test.b
u.r.v.train <- u.r.v[i.train.b,]
u.r.v.test <- u.r.v[i.test.b,]
nb <- naiveBayes(formula, data = u.r.v.train)
new.m <- m
i.test.c <- rep(FALSE,943*1682)
for (j in seq_len(fold.size)){
    new.m[u.r.v.test[j,2],u.r.v.test[j,1]]<-0
    i.test.c[943*(u.r.v.test[j,1]-1)+u.r.v.test[j,2]]<-TRUE
}
for (k in seq_len(943)){
    for (l in seq_len(1682)){
        if (new.m[k,l]==0){
            new.m[k,l]<- predict(nb, newdata = cbind(u.item[l,],u.user[k,])) 
        }
    }
}
write.table(new.m,file=paste("out/bayes/nb", toString(i), ".csv", sep = "")) # drops the rownames
write.table(i.test.c,file=paste("out/bayes/nbTestc", toString(i), ".csv", sep = "")) # drops the rownames
write.table(i.test.b,file=paste("out/bayes/nbTestb", toString(i), ".csv", sep = "")) # drops the rownames
m.train <- new.m
m.train <- m.train
m.svd <- svd(m.train)
to.plot<-rbind(to.plot,unlist(lapply(seq(50), new.m.cv)))
    }


In [15]:
to.plot<-apply(to.plot,2,unlist)

In [16]:
sdev <- apply(to.plot,2,sd) 
sdev
avg <- colMeans(to.plot)
x <- 1:length(avg)

[1] 0.007780475 0.009411512 0.008240024 0.009014070 0.008568303 0.008874017
 [7] 0.008689136 0.008757718 0.009303399 0.009754777 0.010494924 0.006502418
[13] 0.007193417 0.008421638 0.007956843 0.008532578 0.008093278 0.008257436
[19] 0.008058503 0.007515166 0.009046854 0.009241416 0.008688153 0.007733103
[25] 0.007266929 0.006417261 0.007629972 0.007366762 0.008132788 0.007455504
[31] 0.007344119 0.007039992 0.007193698 0.007861601 0.008013698 0.007908795
[37] 0.007722330 0.007187867 0.007418969 0.007879765 0.007858603 0.007873459
[43] 0.007850076 0.007762873 0.007846373 0.008406944 0.008327099 0.008229958
[49] 0.008131629 0.007975460

In [18]:
plot(x,colMeans(to.plot),ylim=range(c(avg-sdev, avg+sdev)),pch=19, xlab="Measurements", ylab="Mean +/- SD",
    main="Scatter plot with std.dev error bars"
)
# hack: we draw arrows but with very special "arrowheads"
arrows(x, avg-sdev, x, avg+sdev, length=0.05, angle=90, code=3)

ERROR: Error in png(tf, width, height, "in", pointsize, bg, res, antialias = antialias): unable to start png() device


Plot with title "Scatter plot with std.dev error bars"

In [20]:
avg

[1] 1.0569826 1.0742123 1.0828661 1.0835690 1.0798092 1.0798346 1.0789639
 [8] 1.0760876 1.0750482 1.0734720 1.0725428 1.0573915 1.0528164 1.0474024
[15] 1.0443070 1.0424406 1.0398680 1.0369106 1.0331738 1.0297980 1.0249841
[22] 1.0192970 1.0163958 1.0121198 1.0071097 1.0026982 0.9980980 0.9939750
[29] 0.9893386 0.9862265 0.9831571 0.9797556 0.9759469 0.9725752 0.9696346
[36] 0.9663531 0.9631160 0.9596934 0.9565264 0.9530836 0.9500124 0.9464360
[43] 0.9432830 0.9402251 0.9371459 0.9339875 0.9305434 0.9270433 0.9239249
[50] 0.9208795

In [21]:
mae(m.train[i.test.c],m.na[i.test.c])

[1] 1.16565

In [6]:
table = list()
test = list()

In [7]:
table[[1]] <- read.table(file=paste("naiveBayes1_1.csv", sep = "")) # drops the rownames
table[[2]] <- read.table(file=paste("naiveBayes1_2.csv", sep = "")) # drops the rownames
table[[3]] <- read.table(file=paste("naiveBayes1_3.csv", sep = "")) # drops the rownames
table[[4]] <- read.table(file=paste("naiveBayes1_4.csv", sep = "")) # drops the rownames
table[[5]] <- read.table(file=paste("naiveBayes1_5.csv", sep = "")) # drops the rownames

test[[1]] <- read.table(file=paste("naiveBayesTestc1_1.csv", sep = "")) # drops the rownames
test[[2]] <- read.table(file=paste("naiveBayesTestc1_2.csv", sep = "")) # drops the rownames
test[[3]] <- read.table(file=paste("naiveBayesTestc1_3.csv", sep = "")) # drops the rownames
test[[4]] <- read.table(file=paste("naiveBayesTestc1_4.csv", sep = "")) # drops the rownames
test[[5]] <- read.table(file=paste("naiveBayesTestc1_5.csv", sep = "")) # drops the rownames


In [8]:
for (i in 1:5){
m.train <- data.matrix(table[[i]])
i.test.c <- unlist(test[[i]])
print(mae(m.train[i.test.c],m.na[i.test.c]))
m.svd <- svd(m.train)
to.plot<-rbind(to.plot,unlist(lapply(seq(100), new.m.cv)))
    }
to.plot<-apply(to.plot,2,unlist)

[1] 1.22005


ERROR: Error in mean(abs(m1 - m2), na.rm = T): object 'i.test.b' not found


In [ ]:
sdev <- apply(to.plot,2,sd) 
avg <- colMeans(to.plot)
x <- 1:length(avg)
plot(x,colMeans(to.plot),ylim=range(c(avg-sdev, avg+sdev)),pch=19, xlab="Measurements", ylab="Mean +/- SD",
    main="Scatter plot with std.dev error bars"
)
# hack: we draw arrows but with very special "arrowheads"
arrows(x, avg-sdev, x, avg+sdev, length=0.05, angle=90, code=3)

In [22]:
avg

[1] 1.0569826 1.0742123 1.0828661 1.0835690 1.0798092 1.0798346 1.0789639
 [8] 1.0760876 1.0750482 1.0734720 1.0725428 1.0573915 1.0528164 1.0474024
[15] 1.0443070 1.0424406 1.0398680 1.0369106 1.0331738 1.0297980 1.0249841
[22] 1.0192970 1.0163958 1.0121198 1.0071097 1.0026982 0.9980980 0.9939750
[29] 0.9893386 0.9862265 0.9831571 0.9797556 0.9759469 0.9725752 0.9696346
[36] 0.9663531 0.9631160 0.9596934 0.9565264 0.9530836 0.9500124 0.9464360
[43] 0.9432830 0.9402251 0.9371459 0.9339875 0.9305434 0.9270433 0.9239249
[50] 0.9208795